<a href="https://colab.research.google.com/github/didi64/Colab_Test/blob/main/Classify_Reviews2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
ROOT = '/content/drive/My Drive/CAS_Gregi/'
drive.mount('/content/drive', force_remount=True)

nb_path = '/content/notebooks'
os.symlink(ROOT + 'CO_modules', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [2]:
import pickle
with open(ROOT + 'data/hotelreviews_cleaned_train_test_spell.pkl','rb') as f:
    data_train, data_test = pickle.load(f)

In [3]:
import random
import numpy as np
import tensorflow as tf
from functools import reduce

In [4]:
def fix_shape(text):
    return tf.constant(text, shape = (1,))

def get_dataset(d):
    ''' get dataset from dict {score: list_of_revies,...}'''
    # list of tuples [(score, review),...]
    all_reviews = reduce(lambda x,y: x + y, [[(r, k) for k,v in d.items() for r in v]])
    random.shuffle(all_reviews)
    reviews = tf.constant([r for r,_ in all_reviews], shape=(len(all_reviews), 1))
    gpt_scores  = tf.constant([s - 1 for _, s in all_reviews])

    n = len(reviews)
    ds = tf.data.Dataset.from_tensor_slices((reviews[:n], gpt_scores[:n]))
    return ds

In [5]:
ds_train = get_dataset(data_train)
ds_test = get_dataset(data_test)

In [6]:
all_reviews = reduce(lambda x,y: x + y, [[(r, k) for d in (data_train, data_test) for k,v in d.items() for r in v]])
WORDS = reduce(lambda x,y:x|y, [set(x[0].split()) for x in all_reviews])

In [7]:
len(WORDS)

9033

In [8]:
# standardization: one element of  standardizations or a callable mapping strings to strings
standardizations = [None, "lower_and_strip_punctuation", "lower", "strip_punctuation"]
max_tokens = 10_000
ngrams = None
sequence_length = 200

tv_layer = tf.keras.layers.TextVectorization(
    max_tokens = max_tokens,
    standardize = standardizations[0],
    ngrams=ngrams,
    output_mode='int',
    output_sequence_length = sequence_length,
    pad_to_max_tokens=True,
)

In [9]:
tv_layer.adapt([r[0] for r in all_reviews])
vocab = tv_layer.get_vocabulary()
len(vocab) # max_tokens

9035

In [10]:
tv_layer(data_train[3][30])

<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([  16,    8,  436,   82,  163,    5,  365,  222,   37,   21,    2,
        377,  259,   36,   20,    5,  276,   10,  104, 2640,   21,    2,
        377,  466,   16,   20,  528,    2,  207,   55,    5,  118,  275,
          3,   44,  105,  210,  192, 1695,    6,  316,    8,    2,  115,
          7,   13,    2,   94,   25,   65,  264,   42,  367,    3,  586,
          5,  664,    4,  392,    8,    2,    7,   36,   14,   40, 1155,
        107,  316,   17,  429,   29,  296,    5,  531,    3,    5,  357,
        113,   28,   89,   28,  291,  398,    6, 4099,    5, 1109, 3744,
          2,    7,    4,   52,    3,  722,    3,    2,   41,    4,   98,
         30,  106,  544,    8,    2,   67,    3,  738,  585, 1656,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [11]:
output_dim =  16
em_layer = tf.keras.layers.Embedding(input_dim = max_tokens + 1, output_dim= output_dim)
em_layer(tv_layer('what a hotel'))

<tf.Tensor: shape=(200, 16), dtype=float32, numpy=
array([[ 2.3678217e-02, -2.1151638e-02, -2.3410155e-02, ...,
         4.5185115e-02,  4.9323205e-02,  2.0979334e-02],
       [-2.6473487e-02, -9.7542033e-03,  4.6490718e-02, ...,
         2.7178898e-03,  2.2125531e-02, -1.7344214e-02],
       [-1.5140474e-02, -3.6553599e-02,  5.3130090e-05, ...,
        -7.5159669e-03,  2.5050528e-03,  2.0912077e-02],
       ...,
       [-4.3067422e-02, -3.4635913e-02,  2.3401227e-02, ...,
         4.9287785e-02, -1.7889142e-02, -1.5301332e-03],
       [-4.3067422e-02, -3.4635913e-02,  2.3401227e-02, ...,
         4.9287785e-02, -1.7889142e-02, -1.5301332e-03],
       [-4.3067422e-02, -3.4635913e-02,  2.3401227e-02, ...,
         4.9287785e-02, -1.7889142e-02, -1.5301332e-03]], dtype=float32)>

In [12]:
model = tf.keras.Sequential([
    tv_layer,
    em_layer,
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'relu')])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 200, 16)           160016    
                                                                 
 dropout (Dropout)           (None, 200, 16)           0         
                                                                 
 lstm (LSTM)                 (None, 10)                1080      
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 flatten (Flatten)           (None, 10)                0         
                                                        

In [13]:
metric = tf.keras.metrics.MeanAbsoluteError(name='mean_absolute_error', dtype=None)
loss =  tf.keras.losses.MeanSquaredError()
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=loss,
    metrics=[metric],
)

In [14]:
# batch train and validate date if this not already happend
if next(iter(ds_train))[0].shape[0] == 1:
    ds_train = ds_train.batch(32)

if next(iter(ds_test))[0].shape[0] == 1:
    ds_test = ds_test.batch(32)

In [15]:
i = 0
for review, _ in iter(ds_train):
    i += 1
    if (i %10) ==0:
        print(i, end = ', ')
    model(review)

10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 

In [28]:
checkpoint_filepath = ROOT + '/My_Models/lstm10/checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='mean_absolute_error',
    mode='min',
    save_best_only=True)

In [31]:
model.fit(
    ds_train,
    epochs = 1000,
    validation_data = ds_test,
    callbacks=[model_checkpoint_callback]
)

model.save_weights(ROOT + 'data/models/lstm10')

Epoch 1/1000
157/157 [==============================] - 2s 11ms/step - loss: 0.1183 - mean_absolute_error: 0.2278 - val_loss: 0.4383 - val_mean_absolute_error: 0.4272
Epoch 2/1000
157/157 [==============================] - 2s 15ms/step - loss: 0.1137 - mean_absolute_error: 0.2249 - val_loss: 0.4088 - val_mean_absolute_error: 0.4221
Epoch 3/1000
157/157 [==============================] - 2s 14ms/step - loss: 0.1186 - mean_absolute_error: 0.2284 - val_loss: 0.4271 - val_mean_absolute_error: 0.4402
Epoch 4/1000
157/157 [==============================] - 2s 11ms/step - loss: 0.1140 - mean_absolute_error: 0.2267 - val_loss: 0.4275 - val_mean_absolute_error: 0.4315
Epoch 5/1000
157/157 [==============================] - 2s 12ms/step - loss: 0.1160 - mean_absolute_error: 0.2271 - val_loss: 0.4279 - val_mean_absolute_error: 0.4334
Epoch 6/1000
157/157 [==============================] - 2s 12ms/step - loss: 0.1126 - mean_absolute_error: 0.2226 - val_loss: 0.4338 - val_mean_absolute_error: 0.430

In [21]:
next(iter(ds_train))

(<tf.Tensor: shape=(32, 1), dtype=string, numpy=
 array([[b'tv not working and reception did nothing after being informed air con not working so room hotter than hell and view from window was a building site   location was good and room seemed a decent standard'],
        [b'we were disappointed to find that the advertised amenities  such as the gym and pool  were closed during our stay  the elevator was slow and unreliable  the room had outdated decor and uncomfortable actresses  the staff appeared disinterested in providing good service'],
        [b'there did appear to be some sort of problem with a fluctuating air conditioning system in the bedroom   friendlies and professionalism of all members of staff together with the cleanliness of the hotel while not forgetting the buffet full english breakfast'],
        [b'checking was a little long winded with lots of questions to answer the cost of the hotel at     per night room only was excessive   the hotel was very modern and clean we

In [30]:
model.evaluate(ds_test)

32/32 [==============================] - 0s 8ms/step - loss: 0.4180 - mean_absolute_error: 0.4213


[0.4180334210395813, 0.4212656617164612]

In [ ]:
model.load_weights(ROOT + 'data/models/lstm10')


In [ ]:
# save the model
#model.save(ROOT + 'data/models/lstm10.sav')

In [17]:
text = fix_shape('the best hotel i ever stayed at')
model.predict(text)

1/1 [==============================] - 0s 395ms/step


array([[1.9284965]], dtype=float32)

In [22]:
text = '''\
the breakfast was excellent and the room very clean and spacious.
the staff was very friendly.
I can really recommend this place'''
model.predict(fix_shape(text))

1/1 [==============================] - 0s 40ms/step


array([[3.2970164]], dtype=float32)

In [ ]:
text = '''\
the breakfast was excellent and the room very clean and spacious.
the staff was very friendly.
I can relly recommend this place'''
model.predict(fix_shape(text))

In [ ]:
text = '''\
the breakfast was ok, but nothing special.
the staff was very friendly.
But the room we small and not suitable for working'''
model.predict(fix_shape(text))

In [23]:
text = '''\
the breakfast was ok, but nothing special.
the staff was quite rude.
And the room we small and not suitable for working'''
model.predict(fix_shape(text))

1/1 [==============================] - 0s 21ms/step


array([[3.2970164]], dtype=float32)

In [24]:
text = '''\
the breakfast was bad, only bread and coffe.
Checkin took forever.
No roomservice.
The room we small and not suitable for working'''
model.predict(fix_shape(text))

1/1 [==============================] - 0s 36ms/step


array([[1.7132279]], dtype=float32)